In [45]:
import pandas as pd  
import numpy as np

In [46]:
df_bridges = pd.read_excel("../data/raw/BMMS_overview.xlsx")
print(df_bridges.columns)
print(df_bridges.head(3))
print("rows:", len(df_bridges))

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')
  road     km              type  LRPName               name  length condition  \
0   N1  1.800       Box Culvert  LRP001a                  .   11.30         A   
1   N1  4.925       Box Culvert  LRP004b                  .    6.60         A   
2   N1  8.976  PC Girder Bridge  LRP008b  Kanch pur Bridge.  394.23         A   

   structureNr                                           roadName chainage  \
0       117861  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...      1.8   
1       117862  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    4.925   
2       119889  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    8.976   

   width  constructionYear  spans   zone circle     division   sub-division  \
0   19.5       

In [47]:
df_bridges.head()

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,1.800,Box Culvert,LRP001a,.,11.30,A,117861,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,1.8,19.5,2005.0,2.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.702889,90.450389,bcs1
1,N1,4.925,Box Culvert,LRP004b,.,6.60,A,117862,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,4.925,35.4,2006.0,1.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.693611,90.478833,bcs1
2,N1,8.976,PC Girder Bridge,LRP008b,Kanch pur Bridge.,394.23,A,119889,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,NaN,NaN,NaN,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,road_precise
3,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.30,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.88,12.2,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699833,90.530722,bcs1
4,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.30,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.2,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699667,90.530722,bcs1


In [49]:
df_bridges[["km","lat","lon"]].describe()

,km,lat,lon
count,21406.000000,21313.000000,21313.000000
mean,41.217405,23.828505,90.227328
std,78.810955,2.247895,3.364169
min,0.000000,0.000000,0.000000
25%,6.207250,22.942527,89.377806
50%,15.147000,23.823743,90.289194
75%,35.012250,24.720939,91.288417
max,522.718000,91.544194,93.298416


In [51]:
#dit is eig cluster 1: de bridges met sws verkeerde waardes 
# 1) Missing coords (NAN values)
mask_missing = df_bridges["lat"].isna() | df_bridges["lon"].isna() 
#--> oplossing: kijken in de andere data sets of het daar wel in staat? je weet wel weg waar op zou staan

# 2) Zero coords (0 or 0.0)
mask_zero = (df_bridges["lat"] == 0) | (df_bridges["lon"] == 0) 


# 3) Invalid range coords
mask_invalid = (df["lat"] < -90) | (df["lat"] > 90) | (df["lon"] < -180) | (df["lon"] > 180)

print("Missing:", mask_missing.sum())
print("Zero:", mask_zero.sum())
print("Invalid:", mask_invalid.sum())

# (optioneel) laat voorbeelden zien
df.loc[mask_invalid, ["road", "km", "lat", "lon"]].head(10) #dit zijn maar kleine fouten net boven 91 graden niet te veel aandacht aan? 
df.loc[mask_zero, ['']

_IncompleteInputError: incomplete input (32252400.py, line 19)

In [44]:
LAT_MIN, LAT_MAX = 20.5, 26.8
LON_MIN, LON_MAX = 88.0, 92.8

out_country = df[
    (df["lat"].notna()) &
    (df["lon"].notna()) &
    (
        (df["lat"] < LAT_MIN) |
        (df["lat"] > LAT_MAX) |
        (df["lon"] < LON_MIN) |
        (df["lon"] > LON_MAX)
    )
]

print("Outside Bangladesh:", len(out_country)) 


NameError: name 'df' is not defined

In [52]:
#cleaned df = clean csv 
df_cleaned_roads = pd.read_csv("../data/raw/clean.csv") 
df_cleaned_roads.head()   
df_bridges.columns

#match them on road and LRP name

Index(['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition',
       'structureNr', 'roadName', 'chainage', 'width', 'constructionYear',
       'spans', 'zone', 'circle', 'division', 'sub-division', 'lat', 'lon',
       'EstimatedLoc'],
      dtype='object')

In [53]:
df_cleaned_roads.columns

Index(['Unnamed: 0', 'road', 'chainage', 'lrp', 'lat', 'lon', 'gap', 'type',
       'name'],
      dtype='object')

In [54]:
def norm(s):
    return s.astype(str).str.strip().str.upper()

# Normalised keys
df_roads = df_cleaned_roads.copy()
df_br = df_bridges.copy()

df_roads["road_norm"] = norm(df_roads["road"])
df_roads["lrp_norm"]  = norm(df_roads["lrp"])

df_br["road_norm"] = norm(df_br["road"])
df_br["lrp_norm"]  = norm(df_br["LRPName"])

In [32]:
#df_br.describe has: 21407 rows

In [33]:
#df_roads.describe has: #52210 rows

In [55]:
#cluster 2 bridges die niet in road file zitten 
#bridges missing in roads; #dus bridge zegt dat het op bepaalde road ligt maar code komt niet overeen met een 
#LRP code op die road 
bridge_keys = df_br[["road_norm","lrp_norm"]]
road_keys   = df_roads[["road_norm","lrp_norm"]]

bridges_missing_in_roads = bridge_keys.merge(
    road_keys, on=["road_norm","lrp_norm"], how="left", indicator=True
).query("_merge == 'left_only'").drop(columns="_merge")

print("Bridges (BMMS) missing in roads:", len(bridges_missing_in_roads))
bridges_missing_in_roads.head(20)

Bridges (BMMS) missing in roads: 6552


,road_norm,lrp_norm
0,N1,LRP001A
1,N1,LRP004B
11,N1,LRP013B
12,N1,LRP014A
15,N1,LRP018C
17,N1,LRP021C
18,N1,LRP021D
19,N1,LRP021C
21,N1,LRP023D
22,N1,LRP023C


In [ ]:
#Wat ik nu nodig heb: outlier file (groupering dus wat Yoran geinterpoleerd heeft) 
#--> die interpolatie dan aanpssen de lon en lat? in bridge file? Maar wat als ze wel goed stonden in bridge file?

In [ ]:
#ik ga ze nu overlappen 

In [56]:
#eerst ff distance functie hier zetten hebben we later nodig: 
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000  # Straal aarde in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [57]:
#inladen: 
# 1. Laad de SCHONE wegen (output van notebook 1)
# Pas het pad aan naar waar jij je clean.csv hebt opgeslagen
df_roads = pd.read_csv("../data/raw/clean.csv")  

# 2. Laad de RUWE bruggen
df_bridges = pd.read_excel("../data/raw/BMMS_overview.xlsx")

# Zorg dat de koppel-kolommen nummers zijn
df_roads['chainage'] = pd.to_numeric(df_roads['chainage'], errors='coerce')
df_bridges['km'] = pd.to_numeric(df_bridges['km'], errors='coerce')

# Zorg dat coordinaten nummers zijn #cooerce zorgt ervoor dat als iets niet goed staat niet verwijdert wordt maar  
#nan value wordt: dus wellicht later nog kijken!! wlke dit waren en of dit typoos waren 
for c in ['lat', 'lon']:
    df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
    df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

print(f"Wegen geladen: {len(df_roads)}")
print(f"Bruggen geladen: {len(df_bridges)}")

Wegen geladen: 52210
Bruggen geladen: 21407


In [51]:
df_analysis = df_bridges.copy()

# Nieuwe kolommen aanmaken
df_analysis['ideal_lat'] = np.nan
df_analysis['ideal_lon'] = np.nan
df_analysis['offset_meters'] = np.nan
df_analysis['error_flag'] = 'OK' # Startwaarde

# Lijst van bekende wegen
known_roads = df_analysis.loc[df_analysis['road'].isin(unique_roads), 'road'].unique()

print("Bezig met berekenen van offsets (dit duurt even)...")

for road_id in known_roads:
    # 1. Pak de weg data (Cleaned)
    road_data = df_roads[df_roads['road'] == road_id].sort_values('chainage')
    if road_data.empty: continue
        
    # 2. Pak de bruggen op deze weg
    mask = df_analysis['road'] == road_id
    bridge_kms = df_analysis.loc[mask, 'km']
    
    # 3. BEREKEN IDEALE LOCATIE (Interpolatie op chainage)
    # Dit is de "waarheid" volgens de chainage
    ideal_lats = np.interp(bridge_kms, road_data['chainage'], road_data['lat'])
    ideal_lons = np.interp(bridge_kms, road_data['chainage'], road_data['lon'])
    
    df_analysis.loc[mask, 'ideal_lat'] = ideal_lats
    df_analysis.loc[mask, 'ideal_lon'] = ideal_lons
    
    # 4. Check of brug "voorbij" de weg ligt (Clamping nodig?)
    max_len = road_data['chainage'].max()
    clamped_mask = mask & (bridge_kms > max_len)
    df_analysis.loc[clamped_mask, 'is_clamped'] = True

# --- FLAGS TOEWIJZEN ---

# Stap A: Bereken afstand (alleen als GPS bestaat)
# Haversine tussen Huidige GPS en Ideale Locatie
dists = haversine_dist(
    df_analysis['lat'], df_analysis['lon'],
    df_analysis['ideal_lat'], df_analysis['ideal_lon']
)
df_analysis['offset_meters'] = dists

# Stap B: Categorieën bepalen

# 1. Onbekende weg
df_analysis.loc[~df_analysis['road'].isin(unique_roads), 'error_flag'] = 'Unknown Road'

# 2. GPS Missing of in Afrika/Oceaan (Bounding box check)
bad_gps = (df_analysis['lat'].isna()) | (df_analysis['lat'] < 20) | (df_analysis['lat'] > 27)
df_analysis.loc[bad_gps, 'error_flag'] = 'Bad GPS (Africa/Null)'

# 3. Grote Offset (>20000m) - Waarschijnlijk verkeerde weg of slechte GPS
large_offset = (~bad_gps) & (df_analysis['offset_meters'] > 20000) #welke treshold 
df_analysis.loc[large_offset, 'error_flag'] = 'Large Offset (>20000m)'

# 4. Kleine Offset (50-500m) - Onnauwkeurigheid
small_offset = (~bad_gps) & (df_analysis['offset_meters'] >= 50) & (df_analysis['offset_meters'] <= 500)
df_analysis.loc[small_offset, 'error_flag'] = 'Small Offset (50-500m)'

# Print resultaten voor je rapport
print("\n=== DATA QUALITY DIAGNOSE ===")
print(df_analysis['error_flag'].value_counts())
print(f"\nAantal bruggen die voorbij einde weg liggen (Clamped): {df_analysis['is_clamped'].sum()}")

Bezig met berekenen van offsets (dit duurt even)...

=== DATA QUALITY DIAGNOSE ===
error_flag
OK                        12221
Small Offset (50-500m)     8739
Large Offset (>20000m)      312
Bad GPS (Africa/Null)       135
Name: count, dtype: int64

Aantal bruggen die voorbij einde weg liggen (Clamped): 411


In [52]:
import pandas as pd
import numpy as np

# 1. Laden & Schoonmaken


# Strings standaardiseren (BELANGRIJK voor matchen)
# In de road file heet de kolom 'lrp', in BMMS heet hij 'LRPName'
df_roads['road'] = df_roads['road'].astype(str).str.strip().str.upper()
df_roads['lrp'] = df_roads['lrp'].astype(str).str.strip().str.upper()

df_bridges['road'] = df_bridges['road'].astype(str).str.strip().str.upper()
df_bridges['LRPName'] = df_bridges['LRPName'].astype(str).str.strip().str.upper()

# Nummers
for c in ['chainage', 'lat', 'lon']: df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
for c in ['km', 'lat', 'lon']: df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

# --- STAP A: ONDERSCHEID MAKEN TUSSEN FOUTTYPES (Null vs Africa) ---
# We maken specifieke categorieën aan
df_bridges['gps_status'] = 'Valid' # Startwaarde

# 1. Null / NaN (Helemaal geen data)
mask_null = df_bridges['lat'].isna() | df_bridges['lon'].isna()
df_bridges.loc[mask_null, 'gps_status'] = 'Missing (NaN)'

# 2. Nul-waarden (0,0 -> ligt in de oceaan bij Afrika)
mask_zero = (df_bridges['lat'] == 0) & (df_bridges['lon'] == 0)
df_bridges.loc[mask_zero, 'gps_status'] = 'Zero Coordinates (0,0)'

# 3. Afrika / Buitenland (Wel cijfers, maar verkeerde plek)
# Alles buiten Lat 20-27 en Lon 88-93
mask_africa = (
    (df_bridges['gps_status'] == 'Valid') & 
    ((df_bridges['lat'] < 20) | (df_bridges['lat'] > 27) | 
     (df_bridges['lon'] < 88) | (df_bridges['lon'] > 93))
)
df_bridges.loc[mask_africa, 'gps_status'] = 'Out of Bounds (Africa)'

print("=== GPS KWALITEIT ANALYSE ===")
print(df_bridges['gps_status'].value_counts())


# --- STAP B: DE HYBRIDE MATCH (LRP Eerst, dan KM) ---
print("\nStart Hybride Match...")

# We gaan proberen de brug te koppelen aan de Road-data via ROAD + LRP NAAM
# We noemen de kolommen van roads even anders om verwarring te voorkomen
roads_lrp_lookup = df_roads[['road', 'lrp', 'lat', 'lon', 'chainage']].rename(
    columns={'lrp': 'LRPName', 'lat': 'road_lat', 'lon': 'road_lon', 'chainage': 'road_chainage'}
)

# Merge: We plakken de road-info aan de brug VAST als de naam matcht
# 'left' merge betekent: houd alle bruggen, zoek road info erbij
df_merged = df_bridges.merge(roads_lrp_lookup, on=['road', 'LRPName'], how='left')

# Nu hebben we bruggen MET en ZONDER match
mask_match = ~df_merged['road_lat'].isna()
print(f"Aantal bruggen gematcht op LRP Naam (Gouden standaard): {mask_match.sum()}")
print(f"Aantal bruggen NIET gematcht op naam (Terugvallen op KM): {(~mask_match).sum()}")

# --- STAP C: DE LAT/LON BEPALEN ---

df_final = df_merged.copy()
df_final['source_of_fix'] = 'None'
df_final['new_lat'] = np.nan
df_final['new_lon'] = np.nan

# 1. Voor LRP Matches: Gebruik de Road Lat/Lon (Dit is de beste data!)
df_final.loc[mask_match, 'new_lat'] = df_final.loc[mask_match, 'road_lat']
df_final.loc[mask_match, 'new_lon'] = df_final.loc[mask_match, 'road_lon']
df_final.loc[mask_match, 'source_of_fix'] = 'LRP Name Match'

# 2. Voor Niet-Matches: Gebruik Interpolatie op KM (Zoals eerder)
# Dit stukje code is iets trager omdat we moeten loopen
# We doen dit alleen voor de bruggen waar 'new_lat' nog leeg is
missing_mask = df_final['new_lat'].isna()
missing_roads = df_final.loc[missing_mask, 'road'].unique()

for r in missing_roads:
    # Pak road data
    r_data = df_roads[df_roads['road'] == r].sort_values('chainage')
    if r_data.empty: continue
    
    # Pak bruggen op deze weg die nog geen fix hebben
    sub_mask = (df_final['road'] == r) & (df_final['new_lat'].isna())
    kms = df_final.loc[sub_mask, 'km']
    
    # Interpoleer
    df_final.loc[sub_mask, 'new_lat'] = np.interp(kms, r_data['chainage'], r_data['lat'])
    df_final.loc[sub_mask, 'new_lon'] = np.interp(kms, r_data['chainage'], r_data['lon'])
    df_final.loc[sub_mask, 'source_of_fix'] = 'KM Interpolation'

# 3. Checken op verkeerde weg (Analyse vraag van jou)
# Als we een LRP Match hebben, maar de KM in BMMS wijkt enorm af van de Chainage in Roads
# Dan was de Chainage in BMMS fout (typefout in km)!
df_final['km_diff'] = abs(df_final['km'] - df_final['road_chainage'])
chainage_typos = df_final[(df_final['source_of_fix'] == 'LRP Name Match') & (df_final['km_diff'] > 1)]

print(f"\nAantal bruggen waar KM fout was, maar LRP naam ons redde: {len(chainage_typos)}")

=== GPS KWALITEIT ANALYSE ===
gps_status
Valid                     21270
Missing (NaN)                94
Out of Bounds (Africa)       26
Zero Coordinates (0,0)       17
Name: count, dtype: int64

Start Hybride Match...
Aantal bruggen gematcht op LRP Naam (Gouden standaard): 14912
Aantal bruggen NIET gematcht op naam (Terugvallen op KM): 6552

Aantal bruggen waar KM fout was, maar LRP naam ons redde: 2828


In [53]:



# Strings
df_roads['road'] = df_roads['road'].astype(str).str.strip().str.upper()
df_roads['lrp'] = df_roads['lrp'].astype(str).str.strip().str.upper()
df_bridges['road'] = df_bridges['road'].astype(str).str.strip().str.upper()
df_bridges['LRPName'] = df_bridges['LRPName'].astype(str).str.strip().str.upper()

# Nummers
for c in ['chainage', 'lat', 'lon']: df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
for c in ['km', 'lat', 'lon']: df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

print("Start Diagnose & Reparatie...")

# ---------------------------------------------------------
# STAP 1: DE DIAGNOSE (Wat is er mis met de originele data?)
# ---------------------------------------------------------
df_final = df_bridges.copy()
df_final['Original_Issue'] = 'OK' # Startwaarde

# A. Check op Null / Zero / Afrika
mask_null = df_final['lat'].isna() | df_final['lon'].isna()
df_final.loc[mask_null, 'Original_Issue'] = 'Missing Coords'

mask_zero = (df_final['lat'] == 0) & (df_final['lon'] == 0)
df_final.loc[mask_zero, 'Original_Issue'] = 'Zero Coords'

# Alles buiten Lat 20-27 en Lon 88-93 is Afrika/Oceaan
mask_africa = (
    (df_final['Original_Issue'] == 'OK') & 
    ((df_final['lat'] < 20) | (df_final['lat'] > 27) | 
     (df_final['lon'] < 88) | (df_final['lon'] > 93))
)
df_final.loc[mask_africa, 'Original_Issue'] = 'Africa/Ocean'

# B. Check op Offsets (Verkeerde weg / Slechte GPS)
# We berekenen waar de brug ZOU moeten zijn volgens KM (Interpolatie)
# Dit is puur voor de diagnose, de fix doen we straks pas definitief.
df_final['chainage_lat'] = np.nan
df_final['chainage_lon'] = np.nan

unique_roads = df_final['road'].unique()
for r in unique_roads:
    r_data = df_roads[df_roads['road'] == r].sort_values('chainage')
    if r_data.empty: continue
    
    mask = df_final['road'] == r
    kms = df_final.loc[mask, 'km']
    
    # Bereken theoretische positie
    df_final.loc[mask, 'chainage_lat'] = np.interp(kms, r_data['chainage'], r_data['lat'])
    df_final.loc[mask, 'chainage_lon'] = np.interp(kms, r_data['chainage'], r_data['lon'])

# Bereken afstand (Haversine) voor diagnose
def haversine(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Alleen checken als GPS geldig is en we de weg kennen
mask_calc = (df_final['Original_Issue'] == 'OK') & (~df_final['chainage_lat'].isna())
dists = haversine(
    df_final.loc[mask_calc, 'lat'], df_final.loc[mask_calc, 'lon'],
    df_final.loc[mask_calc, 'chainage_lat'], df_final.loc[mask_calc, 'chainage_lon']
)
df_final.loc[mask_calc, 'dist_check'] = dists

# Markeer grote en kleine offsets
large_offset = (df_final['Original_Issue'] == 'OK') & (df_final['dist_check'] > 500)
df_final.loc[large_offset, 'Original_Issue'] = 'Large Offset (>500m)'

small_offset = (df_final['Original_Issue'] == 'OK') & (df_final['dist_check'] >= 50) & (df_final['dist_check'] <= 500)
df_final.loc[small_offset, 'Original_Issue'] = 'Small Offset (50-500m)'

# Check Unknown Road
mask_unknown = ~df_final['road'].isin(df_roads['road'])
df_final.loc[mask_unknown, 'Original_Issue'] = 'Unknown Road'

print("Diagnose Voltooid! Dit zijn de problemen in de originele file:")
print(df_final['Original_Issue'].value_counts())



Start Diagnose & Reparatie...
Diagnose Voltooid! Dit zijn de problemen in de originele file:
Original_Issue
Small Offset (50-500m)    8739
OK                        6452
Large Offset (>500m)      5591
Unknown Road               552
Missing Coords              30
Africa/Ocean                26
Zero Coords                 17
Name: count, dtype: int64


In [ ]:
#here we added how often the chainage is wrong, so if lon and lat are on the road but the km is wrong, as that km does not belong to that lon and lat 
#from that followed that the chainge is more wrong so that ofthen the chaina
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree



# Strings standaardiseren (Hoofdletters, geen spaties)
df_roads['road'] = df_roads['road'].astype(str).str.strip().str.upper()
df_bridges['road'] = df_bridges['road'].astype(str).str.strip().str.upper()

# Nummers forceren
for c in ['chainage', 'lat', 'lon']: df_roads[c] = pd.to_numeric(df_roads[c], errors='coerce')
for c in ['km', 'lat', 'lon']: df_bridges[c] = pd.to_numeric(df_bridges[c], errors='coerce')

# =============================================================================
# 2. BASIS DIAGNOSE (Null, Nul, Afrika)
# =============================================================================
print("Basis diagnose uitvoeren...")
df_final = df_bridges.copy()
df_final['Issue_Type'] = 'OK' # We gaan ervan uit dat het goed is, tot tegendeel bewezen

# A. Missing Coordinates
mask_null = df_final['lat'].isna() | df_final['lon'].isna()
df_final.loc[mask_null, 'Issue_Type'] = 'Missing Coords (NaN)'

# B. Zero Coordinates (0,0)
mask_zero = (df_final['lat'] == 0) & (df_final['lon'] == 0)
df_final.loc[mask_zero, 'Issue_Type'] = 'Zero Coords (0,0)'

# C. Africa / Out of Bounds
# Alles buiten Bangladesh (Lat 20-27, Lon 88-93)
mask_africa = (
    (df_final['Issue_Type'] == 'OK') & 
    ((df_final['lat'] < 20) | (df_final['lat'] > 27) | 
     (df_final['lon'] < 88) | (df_final['lon'] > 93))
)
df_final.loc[mask_africa, 'Issue_Type'] = 'Africa/Ocean (GPS Error)'

# D. Unknown Road (Weg bestaat niet in onze clean.csv)
mask_unknown = ~df_final['road'].isin(df_roads['road'])
df_final.loc[mask_unknown, 'Issue_Type'] = 'Unknown Road (Cannot Check)'

# =============================================================================
# 3. CONSISTENTIE CHECK (Chainage vs GPS)
# =============================================================================
print("Chainage vs GPS checken...")

# We berekenen waar de brug ZOU moeten zijn volgens het paaltje (KM)
df_final['ideal_lat'] = np.nan
df_final['ideal_lon'] = np.nan

# Loopje per weg (Interpolatie)
unique_roads = df_final.loc[df_final['Issue_Type'] == 'OK', 'road'].unique()

for r in unique_roads:
    r_data = df_roads[df_roads['road'] == r].sort_values('chainage')
    if r_data.empty: continue
    
    mask = (df_final['road'] == r) & (df_final['Issue_Type'] == 'OK')
    kms = df_final.loc[mask, 'km']
    
    df_final.loc[mask, 'ideal_lat'] = np.interp(kms, r_data['chainage'], r_data['lat'])
    df_final.loc[mask, 'ideal_lon'] = np.interp(kms, r_data['chainage'], r_data['lon'])

# Haversine Functie (Afstand in meters)
def haversine_np(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Bereken afstand tussen GPS en Chainage-Punt
mask_calc = (df_final['Issue_Type'] == 'OK') & (~df_final['ideal_lat'].isna())
dists = haversine_np(
    df_final.loc[mask_calc, 'lat'], df_final.loc[mask_calc, 'lon'],
    df_final.loc[mask_calc, 'ideal_lat'], df_final.loc[mask_calc, 'ideal_lon']
)
df_final.loc[mask_calc, 'dist_offset'] = dists

# E. Markeer Offsets
# Kleine offset (50m - 500m) -> Waarschijnlijk GPS onnauwkeurigheid
mask_small = (df_final['Issue_Type'] == 'OK') & (df_final['dist_offset'] >= 50) & (df_final['dist_offset'] <= 500)
df_final.loc[mask_small, 'Issue_Type'] = 'Small Offset (50-500m)'

# Grote offset (> 500m) -> Hier gaan we dieper op in!
mask_large = (df_final['Issue_Type'] == 'OK') & (df_final['dist_offset'] > 500)
df_final.loc[mask_large, 'Issue_Type'] = 'Large Offset (>500m)'

# =============================================================================
# 4. GEAVANCEERDE ANALYSE (Waarom is de offset groot?)
# =============================================================================
print("Geavanceerde analyse voor grote afwijkingen...")

# We pakken alleen de probleemgevallen
problem_bridges = df_final[df_final['Issue_Type'] == 'Large Offset (>500m)'].copy()

if not problem_bridges.empty:
    # 1. Bouw een zoekboom van ALLE punten in het wegennetwerk
    # (Dit stelt ons in staat om razendsnel de dichtstbijzijnde weg te vinden)
    valid_road_points = df_roads.dropna(subset=['lat', 'lon'])
    road_coords = valid_road_points[['lat', 'lon']].values
    road_names = valid_road_points['road'].values
    
    tree = cKDTree(road_coords)
    
    # 2. Zoek voor elke probleembrug het dichtstbijzijnde punt in HEEL Bangladesh
    bridge_coords = problem_bridges[['lat', 'lon']].values
    dists_deg, indices = tree.query(bridge_coords)
    
    # Afstand omrekenen naar meters (1 graad lat is ong 111km)
    dists_meters = dists_deg * 111000
    nearest_roads = road_names[indices]
    
    # 3. Classificeren
    # F. Wrong Chainage: Brug ligt wél dichtbij (<200m) zijn EIGEN weg, maar chainage klopt niet
    mask_wrong_km = (dists_meters < 200) & (nearest_roads == problem_bridges['road'])
    
    # G. Likely Wrong Road: Brug ligt dichtbij (<200m) een ANDERE weg
    mask_wrong_road = (dists_meters < 200) & (nearest_roads != problem_bridges['road'])
    
    # H. Nowhere: Brug ligt nergens in de buurt (>200m) van welke weg dan ook
    mask_nowhere = (dists_meters >= 200)
    
    # Update de labels in de hoofd-tabel
    # We gebruiken de index om de juiste rijen te updaten
    df_final.loc[problem_bridges[mask_wrong_km].index, 'Issue_Type'] = 'Wrong Chainage (On Road, but KM mismatch)'
    df_final.loc[problem_bridges[mask_wrong_road].index, 'Issue_Type'] = 'Likely Wrong Road (Near other road)'
    df_final.loc[problem_bridges[mask_nowhere].index, 'Issue_Type'] = 'GPS Error (Nowhere near any road)'

# =============================================================================
# 5. RESULTAAT
# =============================================================================
print("\n=== COMPLETE DATA QUALITY ANALYSE ===")
print(df_final['Issue_Type'].value_counts())

# Optioneel: Opslaan om in Excel te bekijken
# df_final.to_csv('bridges_diagnosed.csv', index=False)

Basis diagnose uitvoeren...
Chainage vs GPS checken...
Geavanceerde analyse voor grote afwijkingen...

=== COMPLETE DATA QUALITY ANALYSE ===
Issue_Type
Small Offset (50-500m)                       8739
OK                                           6452
Wrong Chainage (On Road, but KM mismatch)    3909
GPS Error (Nowhere near any road)            1459
Unknown Road (Cannot Check)                   552
Likely Wrong Road (Near other road)           223
Missing Coords (NaN)                           30
Africa/Ocean (GPS Error)                       26
Zero Coords (0,0)                              17
Name: count, dtype: int64


In [ ]:
#only for the bridgest that do not match on LRP name, I anaLysed what can be wrong

# ---------------------------------------------------------
# STAP 1: VOORBEREIDING
# ---------------------------------------------------------
# Zorg dat je input dataframes goed staan
# We maken kopieën om het origineel niet aan te passen
df_b_analysis = df_bridges.copy()
df_r_analysis = df_roads.copy()

# Uniformeer de kolomnamen en types voor de merge
# In clean.csv heet het 'lrp', in bridges 'LRPName' -> wij maken alles 'LRPName'
df_r_analysis = df_r_analysis.rename(columns={'lrp': 'LRPName'})

# Zorg dat alles strings zijn (voorkomt merge fouten)
df_b_analysis['road'] = df_b_analysis['road'].astype(str)
df_b_analysis['LRPName'] = df_b_analysis['LRPName'].astype(str)
df_r_analysis['road'] = df_r_analysis['road'].astype(str)
df_r_analysis['LRPName'] = df_r_analysis['LRPName'].astype(str)

# ---------------------------------------------------------
# STAP 2: DE SPLITSING (MATCH VS NO-MATCH)
# ---------------------------------------------------------
# We mergen om te zien wie er wel/niet matcht
# We halen alleen 'lat' op uit road file om te checken of er een match is
df_merged_check = pd.merge(
    df_b_analysis,
    df_r_analysis[['road', 'LRPName', 'lat']],
    on=['road', 'LRPName'],
    how='left',
    suffixes=('', '_check')
)

# Filter: De unmatched groep (waar lat_check leeg is)
mask_unmatched = df_merged_check['lat_check'].isna()
df_unmatched = df_merged_check[mask_unmatched].copy()

# ---------------------------------------------------------
# STAP 3: ANALYSE - WAAROM GEEN MATCH?
# ---------------------------------------------------------
# Lijst van alle wegen die we WEL kennen in de clean file
known_roads = set(df_r_analysis['road'].unique())

def diagnose_reason(row):
    # 1. Bestaat de weg überhaupt in onze clean file?
    if row['road'] not in known_roads:
        return "Road Not In Clean File"
    
    # 2. Is de LRP naam leeg of nan?
    if row['LRPName'] == 'nan' or row['LRPName'] == '' or pd.isna(row['LRPName']):
        return "LRP Name Missing"
    
    # 3. Weg bestaat wel, LRP naam is er wel, maar staat niet in de road file
    return "LRP Not In Road File"

print("Bezig met diagnosticeren van unmatched bridges...")
df_unmatched['reason'] = df_unmatched.apply(diagnose_reason, axis=1)

# ---------------------------------------------------------
# STAP 4: ANALYSE - HOE SLECHT ZIJN DE ORIGINELE COÖRDINATEN?
# ---------------------------------------------------------
# Grenswaarden voor Bangladesh (ongeveer)
min_lat, max_lat = 20.5, 26.7
min_lon, max_lon = 88.0, 92.9

def check_coords(row):
    lat = row['lat']
    lon = row['lon']
    
    # Check op NaN
    if pd.isna(lat) or pd.isna(lon):
        return "Missing Coordinates (NaN)"
    
    # Check op 0,0
    if lat == 0 or lon == 0:
        return "Zero Coordinates (0,0)"
    
    # Check op Buitenland
    if not (min_lat <= lat <= max_lat) or not (min_lon <= lon <= max_lon):
        return "Outside Bangladesh (Africa/Ocean)"
    
    # Als het hier komt, lijkt het binnen BD te liggen
    return "Coordinates seem OK (Inside BD)"

df_unmatched['coord_quality'] = df_unmatched.apply(check_coords, axis=1)

# ---------------------------------------------------------
# STAP 5: PRINT DE RESULTATEN VOOR JE RAPPORT
# ---------------------------------------------------------
print("\n" + "="*50)
print("ANALYSE VAN DE NIET-GEMATCHTE BRUGGEN")
print("="*50)

print(f"Totaal aantal bruggen: {len(df_merged_check)}")
print(f"Aantal unmatched:      {len(df_unmatched)} ({len(df_unmatched)/len(df_merged_check):.1%})")

print("\n1. WAAROM GEEN MATCH?")
print("-" * 30)
print(df_unmatched['reason'].value_counts())
print("\n> UITLEG:")
print("> 'LRP Not In Road File': De weg is bekend, dus deze kunnen we fixen met interpolatie (chainage).")
print("> 'Road Not In Clean File': Deze wegen missen in de road-dataset. Deze bruggen kunnen we NIET fixen.")

print("\n2. HOE ZIET DE DATA VAN DEZE GROEP ERUIT?")
print("-" * 30)
print(df_unmatched['coord_quality'].value_counts())

print("\n3. KRUISTABEL: REDEN vs KWALITEIT")
print("-" * 30)
print(pd.crosstab(df_unmatched['reason'], df_unmatched['coord_quality']))

Bezig met diagnosticeren van unmatched bridges...

ANALYSE VAN DE NIET-GEMATCHTE BRUGGEN
Totaal aantal bruggen: 21464
Aantal unmatched:      6568 (30.6%)

1. WAAROM GEEN MATCH?
------------------------------
reason
LRP Not In Road File      6016
Road Not In Clean File     552
Name: count, dtype: int64

> UITLEG:
> 'LRP Not In Road File': De weg is bekend, dus deze kunnen we fixen met interpolatie (chainage).
> 'Road Not In Clean File': Deze wegen missen in de road-dataset. Deze bruggen kunnen we NIET fixen.

2. HOE ZIET DE DATA VAN DEZE GROEP ERUIT?
------------------------------
coord_quality
Coordinates seem OK (Inside BD)      6440
Missing Coordinates (NaN)              94
Outside Bangladesh (Africa/Ocean)      17
Zero Coordinates (0,0)                 17
Name: count, dtype: int64

3. KRUISTABEL: REDEN vs KWALITEIT
------------------------------
coord_quality           Coordinates seem OK (Inside BD)  \
reason                                                    
LRP Not In Road File 

In [58]:

#analysis of the bridges with a road name that is missing from the road file 
# 1. STANDAARDISEER NAMEN (PREPARE COLUMNS)
df_bridges['road_norm'] = df_bridges['road'].astype(str).str.strip().str.upper()
df_bridges['lrp_norm'] = df_bridges['LRPName'].astype(str).str.strip().str.upper()

df_roads['road_norm'] = df_roads['road'].astype(str).str.strip().str.upper()
# Check even hoe de kolom heet in jouw clean file (soms 'lrp', soms 'LRPName')
if 'lrp' in df_roads.columns:
    df_roads['lrp_norm'] = df_roads['lrp'].astype(str).str.strip().str.upper()
else:
    df_roads['lrp_norm'] = df_roads['LRPName'].astype(str).str.strip().str.upper()

# 2. IDENTIFY UNMATCHED BRIDGES
merged = df_bridges.merge(
    df_roads[['road_norm', 'lrp_norm']], 
    on=['road_norm', 'lrp_norm'], 
    how='left', 
    indicator=True
)

# Filter: Pak alleen de bruggen die NIET in de road file zitten
unmatched = merged[merged['_merge'] == 'left_only'].copy()

# 3. FILTER ALLEEN OP "ROAD MISSING"
# We maken een set van bekende wegen uit de clean file
known_roads = set(df_roads['road_norm'].unique())

# We pakken nu direct de subset waar de weg NIET in die set zit
missing_roads_df = unmatched[~unmatched['road_norm'].isin(known_roads)].copy()

# 4. BEPAAL WEGTYPE (N, R, Z)
# Pak de eerste letter van de wegnaam
missing_roads_df['road_type'] = missing_roads_df['road_norm'].str[0]

# 5. PRINT RESULTAAT
print(f"Totaal amount of bridges on missing roads: {len(missing_roads_df)}")
print("-" * 40)
print("WELKE TYPEN WEGEN MISSEN WE? (Verdeling N, R, Z)")
print(missing_roads_df['road_type'].value_counts())

print("-" * 40)

#this shows that the bridges on missing roads are mostly Z ways (so local ways), and some Regional roads but not large important 
#national highways where a lot of traffic is likely to use the bridge 


Totaal amount of bridges on missing roads: 552
----------------------------------------
WELKE TYPEN WEGEN MISSEN WE? (Verdeling N, R, Z)
road_type
Z    402
R    150
Name: count, dtype: int64
----------------------------------------


In [59]:
#Solution algorithm wrong


# ---------------------------------------------------------
# STAP 1: LADEN EN STANDAARDISEREN
# ---------------------------------------------------------
def normalize_text(df, col_name):
    return df[col_name].astype(str).str.strip().str.upper()
# Normaliseer de sleutel-kolommen in beide files
df_bridges['road_norm'] = normalize_text(df_bridges, 'road')
df_bridges['lrp_norm'] = normalize_text(df_bridges, 'LRPName')

df_roads['road_norm'] = normalize_text(df_roads, 'road')
# In clean.csv heet de kolom soms 'lrp' of 'LRPName'
if 'lrp' in df_roads.columns:
    df_roads['lrp_norm'] = normalize_text(df_roads, 'lrp')
else:
    df_roads['lrp_norm'] = normalize_text(df_roads, 'LRPName')

# ---------------------------------------------------------
# STAP 2: FILTER ONBEKENDE WEGEN (CLEAN UP)
# ---------------------------------------------------------
print("2. Onbekende wegen verwijderen...")
known_roads = set(df_roads['road_norm'].unique())

# Identificeer bruggen op wegen die niet bestaan in de clean file
mask_unknown = ~df_bridges['road_norm'].isin(known_roads)
df_dropped = df_bridges[mask_unknown].copy()
df_final = df_bridges[~mask_unknown].copy()

print(f"   -> {len(df_dropped)} bruggen verwijderd (weg onbekend in clean dataset).")
print(f"   -> {len(df_final)} bruggen over om te repareren.")

# ---------------------------------------------------------
# STAP 3: VOORBEREIDING INTERPOLATIE (LOOKUP TABELLEN)
# ---------------------------------------------------------
print("3. Lookup tabellen bouwen...")
# We maken dictionaries voor snelheid. Dit is 100x sneller dan steeds filteren.

# A. Max lengte per weg (voor de 'Clamping' check)
road_max_lengths = df_roads.groupby('road_norm')['chainage'].max().to_dict()

# B. De volledige weg data (chainage, lat, lon) gesorteerd
road_dict = {}
for road_name, group in df_roads.groupby('road_norm'):
    # Zorg dat de wegpunten gesorteerd zijn op chainage!
    road_dict[road_name] = group.sort_values('chainage')[['chainage', 'lat', 'lon']]

# ---------------------------------------------------------
# STAP 4: DE GOUDEN MATCH (LRP NAAM)
# ---------------------------------------------------------
print("4. Uitvoeren 'Golden Match' (LRP Naam)...")
# We mergen de clean road coördinaten erbij
df_final = df_final.merge(
    df_roads[['road_norm', 'lrp_norm', 'lat', 'lon']], 
    on=['road_norm', 'lrp_norm'], 
    how='left', 
    suffixes=('', '_matched') # '_matched' kolommen komen uit de road file
)

# ---------------------------------------------------------
# STAP 5: DE REPARATIE FUNCTIE (DE CORE LOGICA)
# ---------------------------------------------------------
print("5. Start reparatie algoritme (Interpolatie & Clamping)...")

def repair_bridge_location(row):
    # SITUATIE A: Golden Match (LRP Naam klopte)
    # -----------------------------------------
    if not pd.isna(row['lat_matched']):
        # We nemen de coördinaten van de weg over
        return row['lat_matched'], row['lon_matched'], "Fixed: Matched on LRP Name"
    
    # SITUATIE B: Geen match, we moeten interpoleren (Chainage)
    # ---------------------------------------------------------
    road_id = row['road_norm']
    km = row['km']
    
    # Error checks
    if pd.isna(km):
        return np.nan, np.nan, "Failed: Missing Chainage"
    
    # Haal referentie data op
    # (We weten zeker dat road bestaat, want stap 2 filterde de rest weg)
    road_data = road_dict[road_id]
    max_len = road_max_lengths[road_id]
    
    # CLAMPING LOGICA (Voorkom dat punten van de kaart vallen)
    used_km = km
    note = "Fixed: Interpolated by Chainage"
    
    if km < 0:
        used_km = 0
        note = "Fixed: Chainage < 0 (Snapped to Start)"
    elif km > max_len:
        used_km = max_len
        note = "Fixed: Chainage > Max Length (Snapped to End)"
        
    # INTERPOLATIE (De Wiskunde)
    # Zoekt de positie van 'used_km' tussen de punten van de weg
    new_lat = np.interp(used_km, road_data['chainage'], road_data['lat'])
    new_lon = np.interp(used_km, road_data['chainage'], road_data['lon'])
    
    return new_lat, new_lon, note

# Pas de functie toe op elke rij
results = df_final.apply(repair_bridge_location, axis=1, result_type='expand')
df_final[['lat_clean', 'lon_clean', 'fix_method']] = results

# ---------------------------------------------------------
# STAP 6: VALIDATIE & OPSCHONEN
# ---------------------------------------------------------
# Bereken hoeveel we de brug verplaatst hebben (als check voor je rapport)
def haversine_dist(lat1, lon1, lat2, lon2):
    R = 6371000
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1-a))

# Alleen berekenen als we originele coords hadden
mask_has_orig = (~df_final['lat'].isna())
df_final.loc[mask_has_orig, 'shift_meters'] = haversine_dist(
    df_final.loc[mask_has_orig, 'lat'], 
    df_final.loc[mask_has_orig, 'lon'], 
    df_final.loc[mask_has_orig, 'lat_clean'], 
    df_final.loc[mask_has_orig, 'lon_clean']
)

# Maak de definitieve export set
# We overschrijven de oude lat/lon met de nieuwe 'clean' versie
df_export = df_final.copy()
df_export['lat'] = df_export['lat_clean']
df_export['lon'] = df_export['lon_clean']

# Selecteer relevante kolommen
export_cols = ['road', 'km', 'type', 'LRPName', 'lat', 'lon', 'fix_method', 'shift_meters', 'condition', 'length']
final_output = df_export[export_cols]

# ---------------------------------------------------------
# STAP 7: RAPPORTAGE
# ---------------------------------------------------------
print("\n" + "="*50)
print("FINAL REPAIR REPORT")
print("="*50)
print(f"Totaal aantal bruggen verwerkt: {len(df_final)}")
print("\n1. GEBRUIKTE METHODES:")
print(df_final['fix_method'].value_counts())

print("\n2. IMPACT VAN DE REPARATIE (VERPLAATSING):")
print(f"Gemiddelde verplaatsing: {df_final['shift_meters'].mean():.1f} meter")
print(f"Mediaan verplaatsing:    {df_final['shift_meters'].median():.1f} meter")

print("\n3. GROTE FOUTEN GECORRIGEERD:")
africa_fixes = (df_final['shift_meters'] > 100000).sum() # > 100km verplaatst
print(f"Aantal bruggen die >100km verplaatst zijn (Africa/Ocean fixes): {africa_fixes}")

# Opslaan
final_output.to_csv("bridges_cleaned_final.csv", index=False)
print("\nBestand opgeslagen als 'bridges_cleaned_final.csv'")

#see how much chainage exceeded, sth it justifys 

#major problems: so dont delete the roads, just leave the info there. 

2. Onbekende wegen verwijderen...
   -> 552 bruggen verwijderd (weg onbekend in clean dataset).
   -> 20855 bruggen over om te repareren.
3. Lookup tabellen bouwen...
4. Uitvoeren 'Golden Match' (LRP Naam)...
5. Start reparatie algoritme (Interpolatie & Clamping)...

FINAL REPAIR REPORT
Totaal aantal bruggen verwerkt: 20912

1. GEBRUIKTE METHODES:
fix_method
Fixed: Matched on LRP Name                       14912
Fixed: Interpolated by Chainage                   5606
Fixed: Chainage > Max Length (Snapped to End)      394
Name: count, dtype: int64

2. IMPACT VAN DE REPARATIE (VERPLAATSING):
Gemiddelde verplaatsing: 17219.7 meter
Mediaan verplaatsing:    101.1 meter

3. GROTE FOUTEN GECORRIGEERD:
Aantal bruggen die >100km verplaatst zijn (Africa/Ocean fixes): 198

Bestand opgeslagen als 'bridges_cleaned_final.csv'


In [ ]:

#This checks the chainage mistakes and further analysis them
# for the roads with wrong chainage, so chainage out of bounds of the road length 
#What the mistake looks like the result shows that it might be filled in in meters instead of KM so /10 error

# ---------------------------------------------------------
# TABEL: ANALYSE VAN CHAINAGE FOUTEN (TOP 20)
# ---------------------------------------------------------

# 1. Bepaal de maximale lengte van elke weg in de clean road file
road_max_lengths = df_roads.groupby('road_norm')['chainage'].max().reset_index()
road_max_lengths.columns = ['road_norm', 'Road_Length_Clean_File']

# 2. Filter de bruggen die we naar het einde hebben getrokken (Snapped to End)
# We gebruiken df_work of df_final uit je vorige stappen
df_errors = df_final[df_final['fix_method'].str.contains("Snapped to End", na=False)].copy()

# 3. Voeg de weglengte toe aan de bruggen-tabel
df_errors = df_errors.merge(road_max_lengths, on='road_norm', how='left')

# 4. Bereken het verschil (de fout)
df_errors['Error_KM'] = df_errors['km'] - df_errors['Road_Length_Clean_File']

# 5. Maak de tabel toonbaar
# We kiezen de belangrijkste kolommen om te inspecteren
tabel_kolommen = [
    'road', 
    'LRPName', 
    'km', 
    'Road_Length_Clean_File', 
    'Error_KM'
]

# Sorteer van hoog naar laag op basis van de fout
fouten_tabel = df_errors[tabel_kolommen].sort_values(by='Error_KM', ascending=False)

print("TOP 20 GROOTSTE CHAINAGE AFWIJKINGEN (in KM):")
print(fouten_tabel.head(60).to_string(index=False))

# Optioneel: Sla deze lijst op om later aan je docent te laten zien
# fouten_tabel.to_csv("chainage_errors_analysis.csv", index=False) 

#It looks like the comma was misplaced so we have to keep that in mind

TOP 20 GROOTSTE CHAINAGE AFWIJKINGEN (in KM):
 road LRPName    km  Road_Length_Clean_File  Error_KM
Z7718 LRP042b 42.35                   8.150    34.200
Z7718 LRP042a 42.22                   8.150    34.070
Z2813 LRP039a 39.18                   5.240    33.940
Z2813 LRP037b 37.81                   5.240    32.570
Z2813 LRP037a 37.43                   5.240    32.190
Z7718 LRP039d 39.58                   8.150    31.430
Z2813 LRP036a 36.40                   5.240    31.160
Z7718 LRP039b 39.28                   8.150    31.130
Z7718 LRP038a 38.93                   8.150    30.780
Z7718 LRP038a 38.93                   8.150    30.780
Z2812 LRP035c 35.23                   5.104    30.126
Z2812 LRP035a 35.12                   5.104    30.016
Z2812 LRP034e 34.98                   5.104    29.876
Z2812 LRP034c 34.64                   5.104    29.536
Z2812 LRP034d 34.29                   5.104    29.186
Z2812 LRP034a 34.01                   5.104    28.906
Z2812 LRP033c 33.62                 

In [61]:
#this shows how often snap to road how often km to meters 
#so with chainage mistake, it can be a small mistake, so like little bit outside the road range, then it does snap to end 
# but when /10 makes more sense (so mistake of meters instead of km, it changes the chainage to meters) 


# ---------------------------------------------------------
# SAFETY CHECK: HEBBEN WE 'GEVAARLIJKE' CORRECTIES?
# ---------------------------------------------------------
print("--- SAFETY CHECK: RATIO ANALYSE ---")

# We gebruiken weer road_max_lengths
road_max_lengths = df_roads.groupby('road_norm')['chainage'].max().to_dict()

def analyze_ratio(row):
    road_id = row['road_norm']
    km = row['km']
    
    if road_id not in road_max_lengths or pd.isna(km): return "Skip"
    max_len = road_max_lengths[road_id]
    
    if km <= max_len: return "OK"
    
    # Bereken de ratio: Hoeveel keer groter is de chainage dan de weg?
    ratio = km / max_len
    
    # Check of /10 zou passen
    fits_div_10 = (km / 10.0) <= max_len
    
    if fits_div_10:
        if ratio < 1.5:
            return "RISICO: Kleine overshoot (Snap to End is beter)"
        else:
            return "VEILIG: Duidelijke Typo (Factor 10+)"
    else:
        return "NO FIT: Ook /10 past niet"

# Voer de check uit op de probleemgevallen
df_safety = df_final[df_final['km'] > 0].copy()
df_safety['Safety_Check'] = df_safety.apply(analyze_ratio, axis=1)

# Filter op de problemen
risk_analysis = df_safety[df_safety['Safety_Check'].str.contains("RISICO|VEILIG")]

print(risk_analysis['Safety_Check'].value_counts())

print("\nVOORBEELDEN VAN RISICO GEVALLEN (Niet delen door 10!):")
print(risk_analysis[risk_analysis['Safety_Check'].str.contains("RISICO")][['road', 'km', 'length']].head())

--- SAFETY CHECK: RATIO ANALYSE ---
Safety_Check
RISICO: Kleine overshoot (Snap to End is beter)    264
VEILIG: Duidelijke Typo (Factor 10+)               147
Name: count, dtype: int64

VOORBEELDEN VAN RISICO GEVALLEN (Niet delen door 10!):
     road       km  length
1704   N7  249.754     2.2
1705   N7  250.336     2.4
1706   N7  250.512     2.2
1707   N7  251.068     2.2
1708   N7  251.434     2.8


In [64]:
#final algorithm that makes clean data set: 
import pandas as pd
import numpy as np


def normalize_text(df, col_name):
    return df[col_name].astype(str).str.strip().str.upper()
# Normaliseer de sleutel-kolommen in beide files
df_bridges['road_norm'] = normalize_text(df_bridges, 'road')
df_bridges['lrp_norm'] = normalize_text(df_bridges, 'LRPName')

df_roads['road_norm'] = normalize_text(df_roads, 'road')
# In clean.csv heet de kolom soms 'lrp' of 'LRPName'
if 'lrp' in df_roads.columns:
    df_roads['lrp_norm'] = normalize_text(df_roads, 'lrp')
else:
    df_roads['lrp_norm'] = normalize_text(df_roads, 'LRPName')



# =========================================================
# STAP 2: WEGEN OPSCHONEN & FILTEREN
# =========================================================
print("2. Onbekende wegen verwijderen en roads opschonen...")

# Sorteer wegen (nodig voor interpolatie)
df_roads = df_roads.sort_values(by=['road_norm', 'chainage'])
df_roads = df_roads.drop_duplicates(subset=['road_norm', 'chainage'], keep='first')

# Filter bruggen op onbekende wegen
known_roads = set(df_roads['road_norm'].unique())
original_count = len(df_bridges)
df_final = df_bridges[df_bridges['road_norm'].isin(known_roads)].copy()

print(f"   -> {original_count - len(df_final)} bruggen verwijderd (weg onbekend).")
print(f"   -> {len(df_final)} bruggen over.")

# =========================================================
# STAP 3: LOOKUP TABELLEN BOUWEN
# =========================================================
print("3. Lookup tabellen bouwen...")
road_max_lengths = df_roads.groupby('road_norm')['chainage'].max().to_dict()

road_dict = {}
for road_name, group in df_roads.groupby('road_norm'):
    road_dict[road_name] = group[['chainage', 'lat', 'lon']]

# =========================================================
# STAP 4: DE GOUDEN MATCH (LRP NAAM)
# =========================================================
print("4. Uitvoeren 'Golden Match' (LRP Naam)...")
df_final = df_final.merge(
    df_roads[['road_norm', 'lrp_norm', 'lat', 'lon']], 
    on=['road_norm', 'lrp_norm'], 
    how='left', 
    suffixes=('', '_matched')
)

# =========================================================
# STAP 5: DE INTELLIGENTE REPARATIE
# =========================================================
print("5. Start reparatie (LRP -> Chainage -> Smart Typo -> Snap)...")

def repair_bridge_location_final(row):
    # 1. LRP MATCH (Goud)
    if not pd.isna(row['lat_matched']):
        return row['lat_matched'], row['lon_matched'], "Fixed: Matched on LRP Name"
    
    road_id = row['road_norm']
    km = row['km']
    
    if pd.isna(km) or road_id not in road_dict:
        return row['lat'], row['lon'], "Error: Data missing"

    road_data = road_dict[road_id]
    max_len = road_max_lengths[road_id]

    # 2. CHAINAGE IS GOED (Zilver)
    if 0 <= km <= max_len:
        new_lat = np.interp(km, road_data['chainage'], road_data['lat'])
        new_lon = np.interp(km, road_data['chainage'], road_data['lon'])
        return new_lat, new_lon, "Fixed: Interpolated by Chainage"

    # 3. CHAINAGE FOUT (Brons / Fallback)
    
    # A. Kleine afwijking (Meetruis) -> Snap to End
    if km < (max_len * 1.1) or (km - max_len) < 1.0:
        lat_end = np.interp(max_len, road_data['chainage'], road_data['lat'])
        lon_end = np.interp(max_len, road_data['chainage'], road_data['lon'])
        return lat_end, lon_end, "Fixed: Snapped to End (Small deviation)"

    # B. Grote afwijking -> Check Typo (Factor 10)
    km_div_10 = km / 10.0
    if km_div_10 <= max_len:
        lat_10 = np.interp(km_div_10, road_data['chainage'], road_data['lat'])
        lon_10 = np.interp(km_div_10, road_data['chainage'], road_data['lon'])
        return lat_10, lon_10, "Fixed: Typo detected (Chainage / 10)"

    # C. Fallback (Nog steeds te groot) -> Snap to End
    used_km = max(0, min(km, max_len)) # Clamp
    new_lat = np.interp(used_km, road_data['chainage'], road_data['lat'])
    new_lon = np.interp(used_km, road_data['chainage'], road_data['lon'])
    return new_lat, new_lon, "Fixed: Snapped to End (Road too short)"

# Voer uit
results = df_final.apply(repair_bridge_location_final, axis=1, result_type='expand')
df_final[['lat_clean', 'lon_clean', 'fix_method']] = results

# =========================================================
# STAP 6: EXPORT NAAR EXCEL
# =========================================================
print(f"6. Opslaan als Excel (Sheet: {SHEET_NAME})...")

original_columns = [
    'road', 'km', 'type', 'LRPName', 'name', 'length', 'condition', 'structureNr', 
    'roadName', 'chainage', 'width', 'constructionYear', 'spans', 'zone', 
    'circle', 'division', 'sub-division', 'lat', 'lon', 'EstimatedLoc'
]

df_export = df_final.copy()
df_export['lat'] = df_export['lat_clean']
df_export['lon'] = df_export['lon_clean']
df_export['EstimatedLoc'] = df_export['fix_method']

# Zorg dat numerieke kolommen schoon zijn
for col in ['lat', 'lon', 'km', 'length']:
    df_export[col] = pd.to_numeric(df_export[col], errors='coerce')

df_export = df_export[original_columns]

# HIER IS DE FIX: sheet_name="BMMS_overview"
df_export.to_excel(OUTPUT_FILE, index=False, sheet_name=SHEET_NAME)

print("\n" + "="*50)
print("SUCCESS!")
print("="*50)
print(df_final['fix_method'].value_counts())
print(f"\nBestand opgeslagen: {OUTPUT_FILE}")
print(f"Tabblad naam: {SHEET_NAME}")



2. Onbekende wegen verwijderen en roads opschonen...
   -> 552 bruggen verwijderd (weg onbekend).
   -> 20855 bruggen over.
3. Lookup tabellen bouwen...
4. Uitvoeren 'Golden Match' (LRP Naam)...
5. Start reparatie (LRP -> Chainage -> Smart Typo -> Snap)...
6. Opslaan als Excel (Sheet: BMMS_overview)...

SUCCESS!
fix_method
Fixed: Matched on LRP Name                 14855
Fixed: Interpolated by Chainage             5610
Fixed: Typo detected (Chainage / 10)         260
Fixed: Snapped to End (Small deviation)      134
Name: count, dtype: int64

Bestand opgeslagen: BMMS_overview_CLEANED.xlsx
Tabblad naam: BMMS_overview
